In [2]:
import math
import numpy as np
import itertools

common.py
read_input function

In [4]:
def read_input(filename):
    with open(filename) as f:
        cities = []
        for line in f.readlines()[1:]:  # Ignore the first line.
            xy = line.split(',')
            cities.append((float(xy[0]), float(xy[1])))
        return cities


def format_tour(tour):
    return 'index\n' + '\n'.join(map(str, tour))


def print_tour(tour):
    print(format_tour(tour))

In [7]:
input0 = read_input("/Users/ada/Desktop/google-step-tsp-main/input_0.csv")
input0

[(214.98279057984195, 762.6903632435094),
 (1222.0393903625825, 229.56212316547953),
 (792.6961393471055, 404.5419583098643),
 (1042.5487563564207, 709.8510160219619),
 (150.17533883877582, 25.512728869805677)]

In [8]:
input1 = read_input("/Users/ada/Desktop/google-step-tsp-main/input_1.csv")
input2 = read_input("/Users/ada/Desktop/google-step-tsp-main/input_2.csv")
input3 = read_input("/Users/ada/Desktop/google-step-tsp-main/input_3.csv")
input4 = read_input("/Users/ada/Desktop/google-step-tsp-main/input_4.csv")
input5 = read_input("/Users/ada/Desktop/google-step-tsp-main/input_5.csv")
input6 = read_input("/Users/ada/Desktop/google-step-tsp-main/input_6.csv")

input_list = [input0, input1, input2, input3, input4, input5, input6]

Calculte the distance!

In [9]:
def distance(city1, city2):
    return math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)

BRUTE FORCE APPROACH: just go through all possibly combinations

In [7]:
def tsp_brute(cities):
    num_cities = len(cities)
    best_distance = float("inf")
    best_solution = None
    
    #generate list of possible routes to visit
    possible_routes = list(itertools.permutations(range(1, num_cities)))
    for route in possible_routes:
        route = [0] + list(route)
        current_distance = 0
        cur_stop = route[0]

        for next_stop in route:
            current_distance += distance(cities[cur_stop], cities[next_stop])
            cur_stop = next_stop
        current_distance += distance(cities[0], cities[cur_stop])
        if  best_distance > current_distance:
            best_distance = current_distance
            best_solution = route
            
        # print(best_solution, best_distance)
    return best_solution, best_distance


In [8]:
tsp_brute(input0)

([0, 3, 1, 2, 4], 3291.6217214092458)

In [9]:
tsp_brute(input1)

([0, 4, 2, 6, 1, 5, 3, 7], 3778.715416492538)

TSP with Simulated Annealing:

SOURCE: https://medium.com/@francis.allanah/travelling-salesman-problem-using-simulated-annealing-f547a71ab3c6 

The SA algorithm simulates the physical process of solid annealing. In this process, a material such as metal or glass is raised to a high temperature and then left to cool, allowing the local regions of order to grow outward. It is a metaheuristic algorithm. 

Input: 
- points: a list containing coordinates of the cities
- temperature: the initial tempurature for the simulated annlealing
- cooling rate: the rate at which the tempuratre drops
- iterations: the number of iterations to occur

Here, we take a the solution from the greedy algothrm as the current solution and calculate the distance. We start the annealing process for the number of iterations. In each iteration, we create a new solution by randomly swapping two cites and then calculating their distance. If this has a smaller distance, it is a better solution. If it is worse, there is a chance it might be accepted depending on its previous acceptance rate and what the current temperature is. 

The acceptance probabilty is the exponential function of the ratio of the difference between the two solutions and the current temperature.

Ada's Opinion: This algorithm depends on many random factors so it might not be good for a small number of interations and would instead work well with a number of trails. It might not be the best for large data sets then. 

GREEDY SOLUTION

In [70]:
def solve_greedy(cities):
    N = len(cities)

    dist = [[0] * N for i in range(N)]
    for i in range(N):
        for j in range(i, N):
            dist[i][j] = dist[j][i] = distance(cities[i], cities[j])

    current_city = 0
    unvisited_cities = set(range(1, N))
    tour = [current_city]

    while unvisited_cities:
        next_city = min(unvisited_cities,
                        key=lambda city: dist[current_city][city])
        unvisited_cities.remove(next_city)
        tour.append(next_city)
        current_city = next_city
    return tour

solve_greedy(input0)

[0, 3, 1, 2, 4]

In [50]:
def tsp_simulated_annealing(input_points, temp, cool_rate, iterations):
    num_points = len(input_points)
    cur_solution = solve_greedy(input_points)   #the current solution can be taken from the greedy solution
    best_solution = cur_solution    #set this to be the best solution as of yet

    current_distance = 0
    best_distance = float("inf")

    for i in range(1, num_points):
        #goes through the current path and its distances from the current solution 
        current_distance += distance(input_points[cur_solution[i-1]], input_points[cur_solution[i]])
    #go full circle start to end
    current_distance += distance(input_points[cur_solution[-1]], input_points[cur_solution[0]])

    for _ in range(iterations):
        temp *= cool_rate       #decrease the temperature 
        better_solution = cur_solution[:]

        #generate random indexes to swap
        index1 = np.random.randint(num_points)
        index2 = np.random.randint(num_points)

        #swap the cities to see if a new tour is better with this set up
        better_solution[index1], better_solution[index2] = better_solution[index2], better_solution[index1]
        # print(cur_solution, better_solution)
        better_distance = 0
        for i in range(1, num_points):
        #goes through the current path and its distances from the current solution 
            better_distance += distance(input_points[cur_solution[i-1]], input_points[cur_solution[i]])
    #go full circle start to end
        better_distance += distance(input_points[cur_solution[-1]], input_points[cur_solution[0]])
        #see if the solution improved
        if better_distance <= current_distance:
            cur_solution = better_solution
            current_distance = better_distance
            if better_distance < best_distance:
                best_distance = better_distance
                better_solution = best_solution
        else: #see if you can use this solution in other iterations even if it sucks now
            prob = np.exp((current_distance-better_distance)/temp)
            if np.random.rand() < prob:
                cur_solution = better_solution
                current_distance = better_distance

    return best_distance, best_solution

tsp_simulated_annealing(input0, 100, 0.99, 100)

(3418.101599132713, [0, 2, 3, 1, 4])

In [51]:
for i in range(len(input_list)):
    best_dist, best_sol = tsp_simulated_annealing(input_list[i], 1000, 0.99, 10000)
    print("The Input for", i, "is:", best_dist, best_sol)

The Input for 0 is: 3291.6217214092458 [0, 2, 3, 1, 4]
The Input for 1 is: 3832.2900939051997 [0, 7, 2, 3, 5, 1, 6, 4]
The Input for 2 is: 5449.435265220031 [0, 8, 7, 14, 2, 3, 5, 11, 1, 6, 9, 15, 12, 10, 13, 4]
The Input for 3 is: 10519.161145182472 [0, 50, 36, 10, 30, 57, 16, 12, 44, 25, 32, 53, 38, 35, 51, 19, 61, 6, 63, 1, 29, 5, 48, 18, 24, 47, 58, 26, 59, 46, 22, 31, 3, 33, 7, 27, 54, 52, 2, 34, 49, 43, 55, 40, 23, 11, 41, 9, 45, 37, 20, 42, 39, 60, 8, 14, 15, 21, 28, 56, 13, 62, 4, 17]
The Input for 4 is: 12684.059709833355 [0, 72, 119, 50, 36, 98, 64, 117, 66, 30, 57, 96, 81, 109, 127, 60, 86, 100, 87, 8, 79, 76, 77, 44, 99, 69, 32, 95, 53, 114, 104, 38, 71, 97, 73, 51, 123, 92, 19, 61, 6, 90, 9, 101, 45, 111, 41, 113, 5, 29, 122, 88, 68, 94, 78, 31, 22, 112, 40, 80, 48, 18, 115, 125, 126, 23, 11, 106, 37, 103, 20, 85, 116, 26, 58, 47, 107, 24, 59, 120, 46, 91, 93, 105, 3, 74, 33, 7, 27, 54, 52, 2, 34, 49, 43, 55, 14, 82, 121, 83, 25, 16, 12, 102, 67, 21, 124, 84, 28, 56, 13, 1

Optimizations

In [1]:
def total_dist(cities):
    total_dist = 0
    num_cities = len(cities)
    for i in range(1, num_cities):
        #goes through the current path and its distances from the current solution 
        total_dist += distance(cities[i-1], cities[i])
    #go full circle start to end
    total_dist += distance(cities[-1], cities[0])
    return total_dist

In [79]:
def one_way(cities):
    n = len(cities)
    solution = total_dist(cities)
    sol = cities
    
    can_be_better = True
    while can_be_better:
        can_be_better = False
        for i in range(1, n-1):
            for j in range(i+1, n):
                new = sol[:]
                new[i:j] = reversed(new[i:j])
                new_dist = total_dist(new)
                if new_dist < solution:
                    solution = new_dist
                    sol = new
                    can_be_better = True
        cities = sol
    return solution

3291.6217214092458

In [80]:
for input in input_list:
    print(one_way(input))

3291.6217214092458
3832.290093905199
5356.030477272383
9887.266289833799
11525.52083879649
22521.70994796096


KeyboardInterrupt: 